In [4]:
!pip install --upgrade pip

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [5]:
!pip install -q awswrangler

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [6]:
import pandas as pd
import numpy as np
import boto3
import sagemaker

sx = model(
image = 'dsfldsf',
entry=''    
)

sx.fit(x_data)


sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-data-prep'

In [7]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt ./

download: s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt to ./churn.txt


In [26]:
df=pd.read_csv('./churn.txt')
df['CustomerID']=df.index
df.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?,CustomerID
0,PA,163,806,403-2562,no,yes,300,8.162204,3,7.579174,...,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,True.,0
1,SC,15,836,158-8416,yes,no,0,10.018993,4,4.226289,...,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,False.,1
2,MO,131,777,896-6253,no,yes,300,4.708490,3,4.768160,...,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,False.,2
3,WY,75,878,817-5729,yes,yes,700,1.268734,3,2.567642,...,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,False.,3
4,WY,146,878,450-4942,yes,no,0,2.696177,3,5.908916,...,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,True.,4


In [9]:
columns_with_nan = ['Account Length', 'CustServ Calls']

In [10]:
df2 = df.copy()
df2[columns_with_nan] = df2[columns_with_nan].mask(np.random.random(df[columns_with_nan].shape) < 5e-2)

In [21]:
df2.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?,CustomerID
0,PA,163.0,806,403-2562,no,yes,300,8.162204,3,7.579174,...,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3.0,True.,0
1,SC,15.0,836,158-8416,yes,no,0,10.018993,4,4.226289,...,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8.0,False.,1
2,MO,131.0,777,896-6253,no,yes,300,4.708490,3,4.768160,...,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4.0,False.,2
3,WY,75.0,878,817-5729,yes,yes,700,1.268734,3,2.567642,...,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6.0,False.,3
4,WY,146.0,878,450-4942,yes,no,0,2.696177,3,5.908916,...,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6.0,True.,4


In [17]:
customer_columns = ['CustomerID', 'State', 'Area Code', 'Phone']
account_columns = ['CustomerID', 'Account Length', "Int'l Plan", 'VMail Plan', 'Churn?']
utility_columns = ['CustomerID', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 
                   'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 
                   'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls']

In [33]:
import awswrangler as wr

databases = wr.catalog.databases()
print(databases)
db_name = 'telco_db'
if db_name not in databases.values:
    wr.catalog.create_database(db_name, description = 'Sample DB for telco churn dataset')
    print(wr.catalog.databases())
else:
    print(f"Database {db_name} already exists")

   Database                        Description
0  telco_db  Sample DB for telco churn dataset
Database telco_db already exists


In [41]:
 df[utility_columns].head()

,CustomerID,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
0,0,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3
1,1,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8
2,2,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4
3,3,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6
4,4,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6


In [43]:
dfs = []
suffix = ['customer_info', 'account_info', 'utility']
for i, columns in enumerate([customer_columns, account_columns, utility_columns]):
    df_tmp = df[columns]
    print(columns)
    df_tmp.head()
    dfs.append(df)
    fname = 'telco_churn_%s' % suffix[i]
    outputpath = f's3://{bucket}/{prefix}/data/{fname}'
    print(outputpath)
    if i > 1:
        wr.s3.to_csv(
            df=df_tmp,
            path = outputpath,
            dataset=True,
            database = db_name,
            table=fname,
            index=False,
            mode = 'overwrite')
    else:
        wr.s3.to_csv(
            df = df_tmp,
            path=f'{outputpath}.csv',
            index=False)

['CustomerID', 'State', 'Area Code', 'Phone']
s3://sagemaker-us-east-1-104877823522/sagemaker-studio-data-prep/data/telco_churn_customer_info
['CustomerID', 'Account Length', "Int'l Plan", 'VMail Plan', 'Churn?']
s3://sagemaker-us-east-1-104877823522/sagemaker-studio-data-prep/data/telco_churn_account_info
['CustomerID', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls']
s3://sagemaker-us-east-1-104877823522/sagemaker-studio-data-prep/data/telco_churn_utility
